In [3]:
import pandas as pd
from transformers import LlamaTokenizer, LlamaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Paso 1: Cargar datos
df = pd.read_csv('/content/mental_health_faq.csv')
questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

# Paso 2: Tokenización
tokenizer = LlamaTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

# Paso 3: Preparación del Dataset
class FAQDataset(Dataset):
    def __init__(self, questions, answers, tokenizer, max_len):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, index):
        question = self.questions[index]
        answer = self.answers[index]

        encoding = self.tokenizer(
            question,
            text_pair=answer,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        labels = encoding['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

dataset = FAQDataset(questions, answers, tokenizer, max_len=512)

# Paso 4: Entrenamiento
model = LlamaForSequenceClassification.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

# Paso 5: Guardar el modelo
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')

# Paso 6: Configurar el chat para pruebas
class ChatBot:
    def __init__(self, model_path, tokenizer_path):
        self.model = LlamaForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = LlamaTokenizer.from_pretrained(tokenizer_path)

    def ask(self, question):
        inputs = self.tokenizer(question, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
        outputs = self.model(**inputs)
        answer = self.tokenizer.decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
        return answer

chatbot = ChatBot('./trained_model', './trained_model')

# Ejemplo de uso del chat para pruebas
while True:
    question = input("You: ")
    if question.lower() in ['exit', 'quit', 'q']:
        break
    answer = chatbot.ask(question)
    print(f"Bot: {answer}")


OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b-chat-hf'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b-chat-hf' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.